### Using APIs to get data from OpenAQ

- Locations are extracted using (API_citylocations.ipynb)

- We took only 300 locations at a time because API enforces a limit of requests

- For merging all files we used (CSV_Merge.ipynb)

In [1]:
import pandas as pd
import json
import requests

In [2]:
# locations = pd.read_csv('/home/jayanthikishore/Downloads/US_PM25/US_cities_locations.csv')
# locations.head()
locations = pd.read_csv('/Users/preethamvignesh/Desktop/OpenAq_Data/Cities_locations/India_cities_locations_test.csv')
locations.head()

,Unnamed: 0,Country,City,Location,Longitude,Latitude
0,0,IN,Agartala,"Kunjaban, Agartala - Tripura SPCB",91.288736,23.862828
1,1,IN,Agra,"Sector-3B Avas Vikas Colony, Agra - UPPCB",77.962370,27.194120
2,2,IN,Agra,"Manoharpur, Agra - UPPCB",78.019360,27.237110
3,3,IN,Agra,"Sanjay Palace, Agra - UPPCB",78.005981,27.198658
4,4,IN,Ahmedabad,"Rakhial, Ahmedabad - IITM",72.625775,23.016834


In [3]:
locationsList = list(locations['Location'])
#print(locationsList[0:5])

**Set the Base URL**

In [4]:
base_url = "https://api.openaq.org/v1/measurements?"

**Build the Query URL**
- We are downloading pm25 datasets using one country's locations

In [5]:
sum = 0

temp_loc = ''

for loc_name in locationsList[0:]:
    #print(loc_name)
    temp_loc = loc_name
    date_start = "2018-01-01T08:00:00" # 12:00 pm in PDT is 7:00 am in UTC
    date_end = "2021-05-31T08:00:00"

    parameter = "pm25"

    query_url = base_url + "location=" + loc_name + "&date_from=" + date_start + "&date_to=" + date_end + "&parameter=" + parameter + "&limit=100000"

    #print("this is our query url: " + query_url)
    
    
    #Calling the API using request()
    if(not (requests.get(query_url))):
        continue
    results_jsons = requests.get(query_url).json()
    
    
    #JSON to DataFrame
    results_list = [results_json for results_json in results_jsons['results']]
    if len(results_list) == 0:
        #print('yes')
        continue
    results_df = pd.DataFrame(results_list,columns=['location','parameter','date','value','unit','coordinates','country','city'])
    
    #Converting dictionaries to other columns in dataframe in some columns 
        #Drop the column containing a dictionary before converting its dict values into individual columns
    
    #data1 = pd.DataFrame(results_df.coordinates.values.tolist())
    
    res_df = results_df.drop('coordinates', axis=1).join(pd.DataFrame(results_df.coordinates.values.tolist()))
    res_df = res_df.drop('date', axis=1).join(pd.DataFrame(res_df.date.values.tolist()))
    
    #Dropping columns we dont need
    res_df.drop(['parameter','unit'],axis=1,inplace=True)
    
    #rename value to pm25
    res_df.rename(columns = {'value':'pm25'},inplace=True)
    
    #Ordering the columns
#     c_dat = list(res_df.columns)
#     c_dat[0],c_dat[1],c_dat[2],c_dat[3],c_dat[4],c_dat[5],c_dat[6],c_dat[7] = c_dat[2],c_dat[3],c_dat[0],c_dat[5],c_dat[4],c_dat[6],c_dat[7],c_dat[1]
#     res_df = res_df[c_dat]
    #print(sum)
    
    
    loc_name = ''.join(e for e in loc_name if (e != '/'))
    
    res_df.to_csv('/Users/preethamvignesh/Desktop/OpenAq_Data/INData/' + loc_name +'_PM25.csv')
#     res_df.to_csv('/home/jayanthikishore/Downloads/US_PM25/data/' + loc_name +'_PM25.csv')
    #print('/home/jayanthikishore/Downloads/US_PM25/data/' + loc_name +'_PM25.csv')
    sum += 1

#print('Final sum: ',sum)

AttributeError: 'NoneType' object has no attribute 'keys'

In [6]:
print(temp_loc)
print(sum)

Asansol Court Area, Asansol - WBPCB
8
